In [1]:
import os
import base64




from operator import itemgetter
from langchain.schema import format_document

from langchain_community.vectorstores import DocArrayInMemorySearch
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts.prompt import PromptTemplate
from langchain_core.runnables import RunnableParallel, RunnablePassthrough, RunnableLambda
from langchain_openai.chat_models import ChatOpenAI
from langchain_openai.embeddings import OpenAIEmbeddings

from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import AIMessage, HumanMessage, get_buffer_string, BaseMessage
from langchain.memory import ConversationBufferMemory

from langchain_community.vectorstores import FAISS
from langchain_core.messages import HumanMessage, SystemMessage
from pprint import pprint
from generate_model_footprint import get_model_footprint, get_prompt
from analyze_solution_picture import analyze_picture
from generate_recommendations import generate_recommendations

In [2]:
#image_path="C:/Users/MANISHGUPTA/kyndryl/github/solution-sustainability/images/architecture2.png"
#components = analyze_picture(image_path)
#print(components)
#exit()

In [3]:
image_path="C:/Users/MANISHGUPTA/kyndryl/github/solution-sustainability/images/architecture.png"
recommendations = generate_recommendations(image_path)
print(recommendations)


['Analyzing the provided solution architecture', 'there are several areas where the carbon footprint could potentially be high due to the nature of cloud services being used and the data flow patterns. Here are some of the patterns and scenarios that could contribute to a higher carbon footprint:1. **Data Ingestion and Continuous Streaming**: The architecture shows data being ingested from vehicles through IoT Hub and processed by Azure Stream Analytics. If there is a high volume of data being continuously streamed from numerous vehicles', 'this could lead to significant energy consumption due to the constant processing required.2. **Complex Data Storage Solutions**: The architecture includes Azure Cosmos DB', 'Azure SQL', 'and Azure Synapse Analytics for serving storage. Each of these services is designed to handle large amounts of data', 'and depending on the scale and usage patterns', 'they could consume a lot of energy. Over-provisioning or inefficient use of these services could r

In [4]:
pprint(recommendations)

['Analyzing the provided solution architecture',
 'there are several areas where the carbon footprint could potentially be high '
 'due to the nature of cloud services being used and the data flow patterns. '
 'Here are some of the patterns and scenarios that could contribute to a '
 'higher carbon footprint:1. **Data Ingestion and Continuous Streaming**: The '
 'architecture shows data being ingested from vehicles through IoT Hub and '
 'processed by Azure Stream Analytics. If there is a high volume of data being '
 'continuously streamed from numerous vehicles',
 'this could lead to significant energy consumption due to the constant '
 'processing required.2. **Complex Data Storage Solutions**: The architecture '
 'includes Azure Cosmos DB',
 'Azure SQL',
 'and Azure Synapse Analytics for serving storage. Each of these services is '
 'designed to handle large amounts of data',
 'and depending on the scale and usage patterns',
 'they could consume a lot of energy. Over-provisioning or

In [236]:
sentences = ""
for sentence in recommendations:
    sentences = sentences + " " + sentence#.replace(":", "\n").replace("(", "").replace(")", "\n").replace("'", "")
pprint(sentences)

(' Based on the provided solution architecture which appears to be a Microsoft '
 'Azure-based IoT solution for vehicles here are some recommendations to '
 "optimize the architecture for a lower carbon footprint:1. ('Recommendation': "
 "'Implement serverless architectures where possible' 'Justification': "
 "'Serverless services like Azure Functions can scale automatically and only "
 'consume resources when needed reducing idle time and thus energy '
 "consumption.')2. ('Recommendation': 'Optimize data flow to reduce data "
 "transfer' 'Justification': 'Minimizing the amount of data that needs to be "
 'sent across the network can reduce energy consumption. This can be achieved '
 'by using edge computing (Azure IoT Edge) to process data locally at the '
 "service center before sending it to the cloud.')3. ('Recommendation': 'Use "
 "Azure's sustainability dashboard to monitor and optimize resource usage' "
 "'Justification': 'The sustainability dashboard can provide insights into t

In [175]:
def get_promptx(service: str) -> list[BaseMessage]:
    prompt = ChatPromptTemplate.from_messages([
        SystemMessage(
            content=(
                "You are a world-class Azure cloud expert."
            )
        ),
        HumanMessage(
            content=[
                {"type": "text", 
                "text": f"For the Azure service, named: {service}, provide a detailed model / formula of how to compute an estimate of the carbon cloud footprint leveraging the knowledge about the service. Make required assumptions for the input parameters of the model and then provide an estimate of the carbon cloud footprint of the component. Ensure you clearly describe the model parameters, the assumptions on their values, the formula(s), and the computation of the estimate using the formula(s). The parameters of the model / formula must include datacenter PUE and carbon intensity among others. The output must be organized in only and only the following 'Description of model', 'Model Parameters', 'Assumptions', 'Formulas', 'Computation', and 'Conclusion'. The output must be in json format to allow for digital processing",
                },
            ]
        )]
    )
    return prompt

def get_model_footprintx(components):
    llm = ChatOpenAI(temperature=0.3, model="gpt-4-turbo-preview", max_tokens=3000, )
    chain = RunnableLambda(get_prompt) |  llm

    outputs = []
    for component in components:
        print(component)
        output = chain.invoke({'service':component})
        outputs.append(output)
    return outputs

In [203]:
components = ['Azure Sphere', 'IoT Hub', 'Azure Sphere Security Service', 'Azure Stream Analytics', 'Azure Cosmos DB', 'Azure SQL', 'Azure Synapse Analytics', 'Web and mobile apps', 'Power Platform and BI apps', 'API Management', 'Azure IoT Edge', 'Azure Defender for IoT', 'Azure DevOps', 'Azure Monitor', 'Azure Key Vault', 'Azure Active Directory', 'HoloLens']
components = ['Cloud Pub/Sub', 'Cloud Dataflow', 'Cloud Storage', 'Cloud Datastore', 'Cloud Bigtable', 'BigQuery', 'Cloud Dataproc', 'Cloud Datalab', 'App Engine', 'Container Engine', 'Compute Engine']
print("------------------------------------------------------")
test_component = [components[len(components)-1]]
test_component

------------------------------------------------------


['Compute Engine']

In [177]:
get_prompt("Azure Key Vault").format(service="Azure Key Vault")

'System: You are a world-class public and private cloud expert.\nHuman: [{\'type\': \'text\', \'text\': \'For the cloud service, named: Azure Key Vault, provide a detailed model of how to compute an estimate of the carbon cloud footprint (in kg CO2 per month) leveraging the knowledge about the service. Follow the steps provided below to generate an output in json:\\nStep 0: Think about a detailed model for computation of the carbon footprint of the Azure Key Vault. You are not bound by the the examples but here are some example questions to consider: does the service have transactions; does it have to receive data, process data, and return data in these transactions; will there be significant energy consumption in data storage; will the service be running in high availability mode and thus having a higher level of energy consumption; think about the co2_emission_factor relevant in the model; etc etc.\\nStep 1: Identify all the input parameters that will be used for the detailed model t

In [178]:
def get_human_message():
    with open("prompts/human_message_for_model_generator.txt", "r") as f:
        human_message = f.read()
    return human_message

In [179]:
x = get_human_message().format(service='Azure Key Vault')
x

'For the cloud service, named: Azure Key Vault, provide a detailed model of how to compute an estimate of the carbon cloud footprint (in kg CO2 per month) leveraging the knowledge about the service. Follow the steps provided below to generate an output in json:\nStep 0: Think about a detailed model for computation of the carbon footprint of the Azure Key Vault. You are not bound by the the examples but here are some example questions to consider: does the service have transactions; does it have to receive data, process data, and return data in these transactions; will there be significant energy consumption in data storage; will the service be running in high availability mode and thus having a higher level of energy consumption; think about the co2_emission_factor relevant in the model; etc etc.\nStep 1: Identify all the input parameters that will be used for the detailed model thought in Step 0. Ensure that the names are concisely defined. Note there will be only one output parameter

In [204]:
outputs = get_model_footprint(test_component)
#print(outputs)

Compute Engine


In [205]:
outputs[0].content

'```json\n{\n  "Description": "The model for computing the carbon footprint of the Compute Engine service considers the energy consumption of the virtual machines (VMs) based on their size (CPU and memory), the number of hours they run, the energy source of the data center, and the co2 emission factor of the energy source. It also takes into account the data transfer in and out of the VMs, assuming that data transfer has a carbon footprint due to the energy used in transmitting data over the network.",\n  "Model Parameters": {\n    "vm_hours": "Total number of hours all VMs run in a month",\n    "cpu_cores": "Number of CPU cores per VM",\n    "memory_gb": "Amount of memory in GB per VM",\n    "data_transfer_gb": "Total amount of data transferred in and out of the VMs in GB",\n    "energy_efficiency_ratio": "Energy efficiency of the data center, represented as a ratio",\n    "co2_emission_factor": "CO2 emission factor of the energy source, in kg CO2 per kWh",\n    "carbon_footprint": "T

In [182]:
def findnth(haystack, needle, n):
    parts= haystack.split(needle, n+1)
    if len(parts)<=n+1:
        return -1
    return len(haystack)-len(parts[-1])-len(needle)

In [183]:


test_str = outputs[0].content
 
# printing original string
print("The original string is : " + str(test_str))
 
# initializing substrings
sub1 = "```json"
sub2 = "```"
 
# getting index of substrings
idx1 = test_str.find(sub1)
print(idx1)
idx2 = test_str.find(sub2)
idx2 = test_str.replace(sub2, 'XXX', 1).find(sub2)
print(idx2)
 
# length of substring 1 is added to
# get string from next character
res = test_str[idx1 + len(sub1) + 1: idx2]
 
# printing result
print("The extracted string : " + res)

The original string is : ```json
{
  "Description": "The model for computing the carbon footprint of Azure DevOps considers the energy consumption of server operations involved in continuous integration/continuous deployment (CI/CD) pipelines, data storage, and data transfer activities. It factors in the number of builds, storage used, data transferred, and the co2 emission factor of the data centers.",
  "Model Parameters": {
    "builds_per_month": "Number of CI/CD pipeline executions per month",
    "average_build_energy": "Average energy consumed per build",
    "storage_gb": "Amount of data storage used in GB",
    "storage_energy_per_gb": "Energy consumed per GB of stored data per month",
    "data_transfer_gb": "Amount of data transferred in GB",
    "data_transfer_energy_per_gb": "Energy consumed per GB of data transferred",
    "co2_emission_factor": "CO2 emission factor of the data center",
    "carbon_footprint": "Total carbon footprint of the Azure DevOps service in kg CO2 

In [223]:
def calculate_formula(parameters, formula):
    for parameter in parameters.keys():
        formula = formula.replace(parameter, str(parameters[parameter]))
    print(formula)
    value = eval(formula)
    print(value)

In [227]:
import json
x = json.loads(res)
print(x['Formula']['carbon_footprint'])
# replace a substring with another string
calculate_formula(x['Assumptions'], x['Formula']['carbon_footprint'])
print(eval(x['Carbon Footprint']['computation']))

(builds_per_month * average_build_energy + storage_gb * storage_energy_per_gb + data_transfer_gb * data_transfer_energy_per_gb) * co2_emission_factor
(1000 * 0.015 + 500 * 0.002 + 1000 * 0.001) * 0.4
6.800000000000001
6.800000000000001
